# Teacher's Assignment No. 15 - Q3

***Author:*** *Ofir Paz* $\qquad$ ***Version:*** *19.03.2024* $\qquad$ ***Course:*** *22961 - Deep Learning*

Welcome to the third question from the fifth assignment of the course *Deep Learning*. \
In this question we will ...

## Imports

First, we will import the required packages for this assignment
- [pytorch](https://pytorch.org/) - One of the most fundemental and famous tensor handling library.

In [1]:
import torch  # pytorch.
import torch.nn as nn  # neural network module.
import torch.optim as optim  # optimization module.
import torch.nn.functional as F  # functional module.
import numpy as np  # numpy.
import cv2 as cv  # opencv.
import torch.utils.data  # data handling module.
import torchvision
import torchvision.transforms as transforms  # image transformation module.
from torch.utils.data import DataLoader  # data loader.
from torchmetrics import Accuracy  # accuracy metric.
import matplotlib.pyplot as plt  # plotting module.

In [18]:
# Set the memory device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Set the batch size
BATCH_SIZE = 128 if DEVICE == 'cuda' else 64

# Set the number of classes
NUM_CLASSES = 10

In [19]:
# Define the transforms for the dataset
def CV_RESIZE(img):
  return cv.resize(img, (224, 224))

transform = transforms.Compose([
  # transforms.RandomHorizontalFlip(),
  # transforms.RandomRotation(10),
  np.array,
  CV_RESIZE,
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225], inplace=True),
])

train_set = torchvision.datasets.CIFAR10(root='CIFAR10', train=True,
                                        download=True, transform=transform)
valid_set = torchvision.datasets.CIFAR10(root='CIFAR10', train=False,
                                        download=True, transform=transform)

train_loader = DataLoader(train_set, BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_set, BATCH_SIZE, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
resnet18 = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT).to(DEVICE) 

In [5]:
for layer_name, layer in resnet18.named_children():
  for param in layer.parameters():
    param.requires_grad = False

resnet18.fc = nn.Linear(512, NUM_CLASSES).to(DEVICE)

In [6]:
optimizer = optim.Adam(resnet18.fc.parameters(), lr=0.0025)
criterion = nn.CrossEntropyLoss()

In [22]:
def train(model, data_loader, optimizer, criterion, num_epochs=10, print_cost=False, print_stride=1):
  """
  Train the model.
  
  :param data_loader: The data loader.
  :param num_epochs: The number of epochs.
  :param lr: The learning rate.
  :param weight_decay: The weight decay.
  :param print_cost: A flag indicating if the cost should be printed.
  :param print_stride: The stride for printing the cost.
  :return: None.
  """
  
  costs = []
  for epoch in range(num_epochs):
    running_loss = 0.
    for i, mini_batch in enumerate(data_loader):

      # get the inputs; data is a list of [inputs, labels]
      inputs, labels = mini_batch

      inputs = inputs.to(DEVICE)
      labels = labels.to(DEVICE)
      
      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      predicts = model(inputs)
      
      loss = criterion(predicts, labels)
      loss.backward()
      optimizer.step()

      # Calc loss
      lloss = loss.item()
      running_loss += lloss * inputs.size(0)

      if print_cost and i % 4 == 0:
        print(f"\r[epoch: {epoch+1}/{num_epochs} MB: {i+1}/{len(data_loader)//BATCH_SIZE}] Loss: {lloss}", end='')

    epoch_loss = running_loss / (len(data_loader) * BATCH_SIZE)
    costs.append(epoch_loss)
    
    if print_cost and (epoch % print_stride == 0 or epoch == num_epochs - 1):
      print(f"\r[epoch: {epoch+1}/{num_epochs}] Total Loss: {epoch_loss}")

  return costs

In [21]:
optimizer = optim.Adam(resnet18.fc.parameters(), lr=0.00025)
criterion = nn.CrossEntropyLoss()
costs = train(resnet18, train_loader, optimizer, criterion, num_epochs=2, print_cost=True)

[epoch: 1/2] Total Loss: 0.673122293942267397
[epoch: 2/2] Total Loss: 0.6330192591566259125


In [11]:
def evaluate(model, data_loader):
  """
  Evaluate the model.
  
  :param model: The model.
  :param data_loader: The data loader.
  :return: The accuracy
  """

  accuracy_module = Accuracy(task="multiclass", num_classes=NUM_CLASSES)
  all_prob_preditions, all_labels = torch.tensor([]), torch.tensor([])

  with torch.no_grad():
    for inputs, labels in data_loader:
      inputs = inputs.to(DEVICE)
      prob_preditions = model(inputs).cpu()
      all_prob_preditions = torch.cat((all_prob_preditions, prob_preditions))
      all_labels = torch.cat((all_labels, labels))

  accuracy = accuracy_module(all_prob_preditions.argmax(dim=-1), all_labels)
  
  return accuracy

In [24]:
print(f"Accuracy: {evaluate(resnet18, valid_loader):.1%}")

Accuracy: 78.8%
